Try to use guide https://www.kaggle.com/amirrezaeian/time-series-data-analysis-using-lstm-tutorial

In [3]:
import sys 
import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns # used for plot interactive graph. 
from sklearn.model_selection import train_test_split # to split the data into two parts
from sklearn.cross_validation import KFold # use for cross validation
from sklearn.preprocessing import StandardScaler # for normalization
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline # pipeline making
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics # for the check the error and accuracy of the model
from sklearn.metrics import mean_squared_error,r2_score

## for Deep-learing:
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Dropout

## from REFIT tutorial
from lxml import objectify
import pandas as pd
import matplotlib.pyplot as plt
import itertools
%matplotlib inline

In [ ]:
# df = pd.read_csv('../input/household_power_consumption.txt', sep=';', 
#                  parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format=True, 
#                  low_memory=False, na_values=['nan','?'], index_col='dt')

In [4]:
path=r'dataset/REFIT_BUILDING_SURVEY.xml'
tree = objectify.parse(path)
root = tree.getroot()
NS={'a':'http://www.refitsmarthomes.org'}

path=r'dataset/REFIT_TIME_SERIES_VALUES.csv'

try:
    csv
except:
    csv=pd.read_csv(path, index_col=0, parse_dates=[1])

In [115]:
## Get climate data
elements=root.xpath('./a:Stock/a:Climate/a:Sensor/a:TimeSeriesVariable', namespaces=NS)
data_clim = {}
time_clim = []
for e in elements:
    id = e.get('id')
    v = csv.loc[id]
    sensor_name = e.get('variableType')
    if len(time_clim) < 1:
        time_clim = v['dateTime'].tolist()
    data_clim[sensor_name] = v['data'].tolist()
df = pd.DataFrame(data_clim, index=time_clim)
##clean df
df = df.iloc[1::2] ## Some timeSteps are not in gas_meter indexes (15 mins and 45 mins values, 00 and 30 mins are in)

    
## Get meter data
building_num = 1
appliance    = 1 #Number of gas meter
tsv = root.xpath('./a:Stock/a:Building[%d]/a:Meter[%d]/a:Sensor/a:TimeSeriesVariable' % (building_num, appliance), namespaces=NS)
id = tsv[0].get('id')
kek = csv.loc[id]
data_gas = {'gas': kek['data'].tolist()}
time_gas = kek['dateTime'].tolist()
kek = pd.DataFrame(data_gas, index=time_gas)

In [124]:
lastind = time_gas[-1]

Timestamp('2015-05-06 23:30:00')

In [121]:
tmp = pd.concat([df, kek], axis=1)
## clean merging. We don't need data where no weather forecast
tmp = tmp[np.isfinite(tmp['Air temperature'])]

In [123]:
tmp.index


DatetimeIndex(['2013-11-04 09:30:00', '2013-11-04 10:00:00',
               '2013-11-04 10:30:00', '2013-11-04 11:00:00',
               '2013-11-04 11:30:00', '2013-11-04 12:00:00',
               '2013-11-04 12:30:00', '2013-11-04 13:00:00',
               '2013-11-04 13:30:00', '2013-11-04 14:00:00',
               ...
               '2015-06-01 10:00:00', '2015-06-01 10:30:00',
               '2015-06-01 11:00:00', '2015-06-01 11:30:00',
               '2015-06-01 12:00:00', '2015-06-01 12:30:00',
               '2015-06-01 13:00:00', '2015-06-01 13:30:00',
               '2015-06-01 14:00:00', '2015-06-01 14:30:00'],
              dtype='datetime64[ns]', length=27563, freq='30T')